# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd

# загрузка данных из файла
recipes = pd.read_csv('data/recipes_sample.csv')

# выбор 5 случайных рецептов
random_recipes = recipes.sample(n=5)

# вывод информации об id рецепта и времени выполнения
print(random_recipes[['id', 'minutes']].to_string(index=False))


    id  minutes
 83479       75
278808       37
176700       80
 67921       35
228072       15


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [21]:
import csv
import xml.etree.ElementTree as ET

def show_info(recipe_id):
    # читаем данные из csv файла
    with open('data/recipes_sample.csv') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['id'] == str(recipe_id):
                recipe_name = row['name'].title()
                author_id = row['contributor_id']
                cook_time = row['minutes']
                break
    
    # читаем данные из xml файла
    with open('data/steps_sample.xml') as f:
        xml_data = f.read()
        root = ET.fromstring(xml_data)
        steps = root.find(f"./recipe[id='{recipe_id}']/steps")
        step_list = [step.text.strip() for step in steps]

    # формируем строку с описанием рецепта
    recipe_info = f"{recipe_name}\n\n"
    recipe_info += '\n'.join([f"{i+1}. {step}" for i, step in enumerate(step_list)])
    recipe_info += '\n----------\n'
    recipe_info += f"Автор: {author_id}\n"
    recipe_info += f"Среднее время приготовления: {cook_time} минут"

    return recipe_info

# вызываем функцию для рецепта с id 170895 и выводим результат на экран
print(show_info(170895))


Leeks And Parsnips  Sauteed Or Creamed

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an additi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [22]:
import re
import xml.etree.ElementTree as ET

# читаем данные из xml файла
with open('data/steps_sample.xml') as f:
    xml_data = f.read()
    root = ET.fromstring(xml_data)
    steps = root.find("./recipe[id='25082']/steps")
    step_list = [step.text.strip() for step in steps]

# регулярное выражение для поиска шаблона в строке
pattern = r"\d+\s+(hour|hours|minute|minutes)\b"

# производим поиск по данному регулярному выражению в каждом шаге рецепта
for step in step_list:
    matches = re.findall(pattern, step)
    if matches:
        print(matches)


['minutes']
['minutes']
['hours']
['minutes']
['minutes', 'minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [49]:
import pandas as pd
import re

# Чтение данных из файла
recipes = pd.read_csv("data/recipes_sample.csv")

# Преобразование значений в столбце "description" в строки
recipes["description"] = recipes["description"].astype(str)

# Функция для поиска шаблона "this..., but" в тексте описания
def find_pattern(description):
    pattern = r"^this[\w\s]+s*but"
    return re.search(pattern, description)

# Применяем функцию find_pattern ко всем описаниям и сохраняем результаты в новый столбец
recipes["contains_pattern"] = recipes["description"].apply(lambda x: find_pattern(x) is not None)

# Фильтруем рецепты, которые содержат шаблон
matched_recipes = recipes[recipes["contains_pattern"]]

# Выводим количество рецептов, которые содержат шаблон, и несколько примеров описаний
print(f"Количество рецептов, которые содержат шаблон: {len(matched_recipes)}")
print("Примеры описаний:")
for description in matched_recipes["description"].head(3):
    print(description)


Количество рецептов, которые содержат шаблон: 141
Примеры описаний:
this recipe requires a full day of cooking but the effort is worth it.  my mom got this recipe over 50 years ago from an italian friend.  the baking soda is the secret ingredient to making this recipe a success so don't eliminate it.  when finished, it can almost be eaten with a fork.  i quadruple the recipe so i have some to freeze.
this buttery and nutty chocolate confection was inspired by the nuts that fall from ohio's famous buckeye trees. come holiday time, they are produced (and consumed) in mass quantities in "the buckeye state," as well as in neighboring states throughout the midwest. note: you may substitute peanut or cashew butter for almond butter; omit almond extract if you use a substitution.
this is a morroccan recipe for homemade almond butter.  making it yourself lets you control the sodium and oil content.  store in an airtight container for use in place of peanut butter for a change of pace.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [62]:
import pandas as pd
import re

# загрузка данных
recipes_df = pd.read_csv('data/recipes_sample.csv')

# получение текста шагов для рецепта с id 72367
steps = recipes_df.loc[recipes_df['id'] == 72367, 'n_steps'].iloc[0]

# убираем пробелы до и после символа дроби в тексте шагов
steps_cleaned = re.sub(r'(\d+)\s*/\s*(\d+)', r'\1/\2', str(steps))

# выводим очищенные шаги рецепта
print(steps_cleaned)


12.0


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [65]:
pip install nltk


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [66]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/freshovich/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [69]:
import nltk
import csv

recipes = []

with open('data/recipes_sample.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        recipes.append(row)


In [70]:
from nltk.tokenize import word_tokenize

# Получаем список шагов рецептов
steps = [row[6] for row in recipes[1:]]
# Разбиваем шаги рецептов на слова
words = []
for step in steps:
    words.extend(word_tokenize(step))

In [71]:
unique_words = set([word.lower() for word in words if word.isalpha()])
print(f"Количество уникальных слов: {len(unique_words)}")

Количество уникальных слов: 22049


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [78]:
import pandas as pd
import nltk

recipes = pd.read_csv('data/recipes_sample.csv')

# преобразование данных в столбце "description" в строковый тип
recipes['description'] = recipes['description'].astype(str)

# разбиение описаний рецептов на предложения
recipes['sentences'] = recipes['description'].apply(nltk.sent_tokenize)

# получение длины каждого списка предложений
recipes['num_sentences'] = recipes['sentences'].apply(len)

# вывод 5 самых длинных описаний рецептов
longest_desc = recipes.sort_values(by='num_sentences', ascending=False).head(5)['description']
print(longest_desc)


18408    this wonderful icing is used for icing cakes a...
481      a translucent golden-brown crust allows the gr...
22566    this is one of the best soups i've ever made a...
16296    the first time i made this cake i grated a mil...
6779     i wrote this because there are an astounding l...
Name: description, dtype: object


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [79]:
import nltk

def print_pos_info(sentence):
    # Токенизация предложения
    tokens = nltk.word_tokenize(sentence)
    # Определение частей речи для каждого слова
    pos_tags = nltk.pos_tag(tokens)
    # Список кортежей (слово, часть речи)
    word_pos_pairs = [(word.lower(), pos) for word, pos in pos_tags]
    # Вывод информации о частях речи слов
    for word, pos in word_pos_pairs:
        # Ширина ячейки для части речи (4 символа)
        pos_width = 4
        # Вывод слова
        print(f"{word}{' '*(pos_width-len(word))}", end="")
        # Вывод части речи
        print(pos)

In [83]:
import pandas as pd

# Загрузка данных
recipes = pd.read_csv('data/recipes_sample.csv')

# Получение названия рецепта с id 241106
recipe_name = recipes.loc[recipes['id'] == 241106, 'name'].values[0]

# Вывод информации о частях речи в названии рецепта
print_pos_info(recipe_name)

eggplantJJ
steaksNNS
withIN
chickpeasNNS
fetaVBP
cheeseJJ
and CC
blackJJ
olivesNNS


In [82]:
import nltk

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/freshovich/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True